In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from IPython.display import display

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 2
TOTAL_BUDGET = 1000

In [3]:
current_team_names = {"André Onana",
                      "Matz Sels",
                      "Gabriel dos Santos Magalhães",
                      "Michael Keane",
                      "Wout Faes",
                      "Jack Stephens",
                      "Ola Aina",
                      "Morgan Rogers",
                      "Bryan Mbeumo",
                      "Facundo Buonanotte",
                      "Luis Díaz",
                      "Mohamed Salah",
                      "Yoane Wissa",
                      "Erling Haaland",
                      "Chris Wood"
                      }

In [4]:
with open("data/data.csv","r",encoding="utf-8") as f:
    all_player_data = pd.read_csv(f)

In [5]:
current_team = Team.fromNameSet(current_team_names, all_player_data)

13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15


In [6]:
current_team_cost = current_team.getTotalCost()
current_team_cost

1000

In [7]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)
data = all_data["data"][SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame(data=data)

In [8]:
selected_team = Team.fromDataFrame(selected_team_df)

13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15
13
14
15


In [9]:
print(selected_team)


Total Score: 0

Goalkeepers:
- Jordan Pickford (Captain) 	Score: 0	Cost: 49	Fixture Difficulty: 2.5
- Łukasz Fabiański (Vice Captain) 	Score: 0	Cost: 40	Fixture Difficulty: 0.8333333333333334
Defenders:
- Murillo Santiago Costa dos Santos	Score: 0	Cost: 45	Fixture Difficulty: -1.8333333333333333
- Joško Gvardiol	Score: 0	Cost: 63	Fixture Difficulty: -3.5
- Ola Aina	Score: 0	Cost: 48	Fixture Difficulty: -1.8333333333333333
- Virgil van Dijk	Score: 0	Cost: 63	Fixture Difficulty: -4.0
- Jan Bednarek	Score: 0	Cost: 40	Fixture Difficulty: 7.333333333333333
Attackers:
- Chris Wood	Score: 0	Cost: 66	Fixture Difficulty: -1.8333333333333333
- Matheus Santos Carneiro Da Cunha	Score: 0	Cost: 68	Fixture Difficulty: 1.5
- Danny Ings	Score: 0	Cost: 50	Fixture Difficulty: 0.8333333333333334
Midfielders:
- Cole Palmer	Score: 0	Cost: 109	Fixture Difficulty: -4.166666666666667
- Bruno Borges Fernandes	Score: 0	Cost: 84	Fixture Difficulty: 1.6666666666666667
- Bryan Mbeumo	Score: 0	Cost: 79	Fixture Diff

In [10]:
new_players = selected_team - current_team
print(new_players)


Total Score: 0

Goalkeepers:
- Jordan Pickford (Captain) 	Score: 0	Cost: 49	Fixture Difficulty: 2.5
- Łukasz Fabiański (Vice Captain) 	Score: 0	Cost: 40	Fixture Difficulty: 0.8333333333333334
Defenders:
- Murillo Santiago Costa dos Santos	Score: 0	Cost: 45	Fixture Difficulty: -1.8333333333333333
- Joško Gvardiol	Score: 0	Cost: 63	Fixture Difficulty: -3.5
- Virgil van Dijk	Score: 0	Cost: 63	Fixture Difficulty: -4.0
- Jan Bednarek	Score: 0	Cost: 40	Fixture Difficulty: 7.333333333333333
Attackers:
- Matheus Santos Carneiro Da Cunha	Score: 0	Cost: 68	Fixture Difficulty: 1.5
- Danny Ings	Score: 0	Cost: 50	Fixture Difficulty: 0.8333333333333334
Midfielders:
- Cole Palmer	Score: 0	Cost: 109	Fixture Difficulty: -4.166666666666667
- Bruno Borges Fernandes	Score: 0	Cost: 84	Fixture Difficulty: 1.6666666666666667
- Harry Winks	Score: 0	Cost: 45	Fixture Difficulty: 3.0


In [11]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

978

In [12]:
added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

Added cost: -22


In [13]:
surplus = current_team_cost - new_team_cost

In [14]:
HEURISTIC = "total_points"
new_players.calculateScore(HEURISTIC)
selected_team.calculateScore(HEURISTIC)
current_team.calculateScore(HEURISTIC)

In [15]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 940.3666666666668


In [16]:
def too_expensive():
    pass

In [17]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    bestTransfer = max(allTransfers)
    if(bestTransfer.getScoreDif() > 0):
        return bestTransfer
    else:
        return None

In [18]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [19]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [20]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)

    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [21]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team):
    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    return newTeam

In [22]:
newTeam = getNewTeam(current_team, selected_team, new_players)
newTeam = newTeam.toBenchTeam()
print(newTeam)

Best transfer:
Transfer from Erling Haaland -> Matheus Santos Carneiro Da Cunha:
Old player: Erling Haaland	Score: 349.1	Cost: 152	Fixture Difficulty: -3.5
New player: Matheus Santos Carneiro Da Cunha	Score: 532.5	Cost: 68	Fixture Difficulty: 1.5
- Cost change: -84
- Score change: 183.4
# Starting 11:

Total Score: 3779.266666666667

Goalkeepers:
- André Onana	Score: 213.67	Cost: 51	Fixture Difficulty: 1.6666666666666667
Defenders:
- Ola Aina	Score: 286.17	Cost: 48	Fixture Difficulty: -1.8333333333333333
- Gabriel dos Santos Magalhães	Score: 72.67	Cost: 61	Fixture Difficulty: -3.8333333333333335
- Michael Keane	Score: 34.5	Cost: 43	Fixture Difficulty: 2.5
- Jack Stephens	Score: 34.33	Cost: 40	Fixture Difficulty: 7.333333333333333
Attackers:
- Matheus Santos Carneiro Da Cunha	Score: 532.5	Cost: 68	Fixture Difficulty: 1.5
- Chris Wood	Score: 494.57	Cost: 66	Fixture Difficulty: -1.8333333333333333
Midfielders:
- Mohamed Salah (Captain) 	Score: 1173.0	Cost: 129	Fixture Difficulty: -4.0
- B